In [1]:
#!/usr/bin/env python3

import sys
import pandas as pd
import datetime
import requests
import numpy as np
from pandas import ExcelWriter
import os
import glob
import pytz
from dateutil.tz import gettz
import timeit
from time import sleep

# from datetime import datetime
from datetime import timedelta
import time
import matplotlib.pyplot as plt
%matplotlib inline
import pickle

pd.set_option('display.max_rows', 50000)
pd.set_option('display.max_columns', 50000)

In [2]:
phase='A'
mapping = {}
mapping['p'] = 'pwr%s' % phase
mapping['q'] = 'rpwr%s' % phase
mapping['v'] = 'vlt%s' % phase
mapping['i'] = 'cur%s' % phase
mapping['s'] = 'apwr%s' % phase
mapping['f'] = 'freq%s' % phase
mapping['pF'] = 'cos%s' % phase
mapping['cF'] = 'scre%s' % phase
mapping['an'] = 'angle%s' % phase
mapping['cE'] = 'cnrg%s' % phase
mapping['pE'] = 'pnrg%s' % phase
mapping['cR'] = 'crnrg%s' % phase
mapping['pR'] = 'prnrg%s' % phase
mapping['avgSP'] = 'avgSP%s' %phase
mapping['sign'] = 'sign%s' %phase

In [31]:
import ast
file1 = open('myfile.txt', 'r') 
Lines = file1.readlines() 

df = pd.DataFrame([])
# Strips the newline character 
for line in Lines:
#     print(line.rstrip())
    t = line.rstrip()#.split('&')
    t = t.replace(',', '.')
    t = t.split('&')
    if t[0] != '':
        if t[2][0] == 'L':
            t[2] = 'L=' + t[2][1]
        else:
            t[2] = 'P=' + t[2][1]
            t[3] = 'S=' + t[3][1]
    
        mydict = dict(zip((v.split('=')[0] for v in t), ((v.split('=')[1]) for v in t)))
#         mydict = ast.literal_eval(line.strip())
        df = pd.concat([df,pd.DataFrame([mydict])])


In [32]:
for col in df.columns:
    if ((col=='utc') or (col=='sN')):
        df[col] = df[col].astype('int')
    else:
        df[col] = df[col].astype('float')
# df = df.loc[df['p']>=0]
# df = df[['p','q','sN','utc','ms']]
df.reset_index(inplace= True, drop=True)

df

,sN,ms,L,utc,v,i,p,q,s,f,pF,cF,an,cE,pE,cR,pR
0,1919,378817.0,1.0,657369756,238.456,0.018,-0.0,4.1,4.2,50.000,1.00,2.489,103.9,28610.0,327.0,1949.0,0.0
1,1920,378827.0,1.0,657369756,238.455,0.018,-0.1,-3.3,4.2,49.990,1.00,2.489,230.9,28610.0,327.0,1949.0,0.0
2,1921,378837.0,1.0,657369756,238.453,0.018,-0.1,-3.3,4.2,49.990,1.00,2.489,230.9,28610.0,327.0,1949.0,0.0
3,1922,378847.0,1.0,657369756,238.452,0.018,-0.1,-3.3,4.2,49.990,1.00,2.569,230.9,28610.0,327.0,1949.0,0.0
4,1923,378857.0,1.0,657369756,238.451,0.018,-0.1,2.1,4.3,49.990,1.00,2.569,149.7,28610.0,327.0,1949.0,0.0
5,1924,378867.0,1.0,657369756,238.449,0.018,-0.1,-0.3,4.3,50.000,1.00,2.569,184.5,28610.0,327.0,1949.0,0.0
6,1925,378877.0,1.0,657369756,238.448,0.018,-0.1,3.2,4.3,50.000,1.00,2.006,132.0,28610.0,327.0,1949.0,0.0
7,1926,378887.0,1.0,657369756,238.446,0.018,-0.1,1.2,4.3,49.990,1.00,2.006,162.7,28610.0,327.0,1949.0,0.0
8,1927,378897.0,1.0,657369756,238.444,0.018,-0.1,3.9,4.3,49.990,1.00,2.006,67.4,28610.0,327.0,1949.0,0.0
9,1928,378907.0,1.0,657369756,238.441,0.018,-0.1,0.0,4.3,50.000,1.00,2.006,179.5,28610.0,327.0,1949.0,0.0


In [29]:
# f = open("test_drill.txt",'w+')
# for i in range(0,80):
#     tmp = pd.DataFrame([])
#     tmp = df.copy()
#     f.write('iteration for i=%i \n' % i)
#     avP = df['p'].iloc[i:i+20].mean()
#     avR = df['q'].iloc[i:i+20].mean()
#     f.write('stable avg active %f, steady avg reactive %f \n' % (avP,avR))
#     tmp['np'] = np.abs(tmp['p']-avP)
#     tmp['nq'] = np.abs(tmp['q']-avR)
    
    
#     f.write('mean active: %f \n' % np.mean(tmp['np'].iloc[(i+240):(i+280)]))
#     f.write('min active: %f \n' % np.min(tmp['np'].iloc[(i+240):(i+280)]))
#     f.write('max active: %f \n' % np.max(tmp['np'].iloc[(i+240):(i+280)]))
#     f.write('std active: %f \n' % np.std(tmp['np'].iloc[(i+240):(i+280)]))
    
#     f.write('mean reactive: %f \n' % np.mean(tmp['nq'].iloc[(i+240):(i+280)]))
#     f.write('min reactive: %f \n' % np.min(tmp['nq'].iloc[(i+240):(i+280)]))
#     f.write('max reactive: %f \n' % np.max(tmp['nq'].iloc[(i+240):(i+280)]))
#     f.write('std reactive: %f \n'% np.std(tmp['nq'].iloc[(i+240):(i+280)]))
    
#     f.write('************************************************')

In [24]:
file2 = open('myfile2.txt', 'r') 
Lines = file2.readlines() 

df2 = pd.DataFrame([])
# Strips the newline character 
for line in Lines:
#     print(line.rstrip())
    t = line.rstrip()#.split('&')
    t = t.replace(',', '.')
    t = t.split('&')
    if t[0] != '':
        if t[2][0] == 'L':
            t[2] = 'L=' + t[2][1]
        else:
            t[2] = 'P=' + t[2][1]
            t[3] = 'S=' + t[3][1]
    
        mydict = dict(zip((v.split('=')[0] for v in t), ((v.split('=')[1]) for v in t)))
#         mydict = ast.literal_eval(line.strip())
        df2 = pd.concat([df2,pd.DataFrame([mydict])])
    
    
for col in df2.columns:
    if ((col=='utc') or (col=='sN')):
        df2[col] = df2[col].astype('int')
    else:
        df2[col] = df2[col].astype('float')
df2.reset_index(inplace= True, drop=True)

df2.head()

,sN,ms,P,S,avgP,minP,maxP,stdP,avgR,minR,maxR,stdR,ms_S,ms_E
0,2209,381705.0,1.0,1.0,333.8,252.0,373.6,34.0,4.6,2.2,9.3,1.7,380015.0,380405.0
1,2215,381755.0,1.0,2.0,143.3,86.0,244.3,47.0,5.3,2.5,9.1,1.7,380415.0,380805.0
2,2221,381805.0,1.0,3.0,70.1,64.0,84.6,5.9,3.0,0.8,9.0,1.9,380815.0,381205.0
3,2227,381855.0,1.0,4.0,60.2,60.2,63.7,1.0,2.9,2.4,5.6,0.9,381215.0,381605.0


In [33]:
S1 = df.loc[df['ms']>=df2['ms_S'].iloc[0]]
S1 = S1.loc[S1['ms']<=df2['ms_E'].iloc[0]+9]

S2 = df.loc[df['ms']>=df2['ms_S'].iloc[1]]
S2 = S2.loc[S2['ms']<=df2['ms_E'].iloc[1]+9]

S3 = df.loc[df['ms']>=df2['ms_S'].iloc[2]]
S3 = S3.loc[S3['ms']<=df2['ms_E'].iloc[2]+9]

S4 = df.loc[df['ms']>=df2['ms_S'].iloc[3]]
S4 = S4.loc[S4['ms']<=df2['ms_E'].iloc[3]+9]

tmp = df.loc[df['ms']>=378815]
tmp = tmp.loc[tmp['ms']<=379215]

avP = tmp['p'].mean()
avR = tmp['q'].mean()

In [21]:
z = S1['p'].values

sumz2 = 0
sumz = 0
for x in z:
    sumz2 += x**2
    sumz += x
print(S1.shape[0]*sumz2)
standev = np.sqrt((S1.shape[0]*sumz2 - (sumz**2))/S1.shape[0]**2)
print(standev)

50312186.40000002
67.48360004475178


In [26]:
n = S1.shape[0]
x = [2,4,3,1]
def StanD(x,n):
    Std = np.sqrt((n*np.sum(i**2 for i in x) - np.sum(x)**2)/(n**2))
    return Std

# print(StanD(S1['p'].values))
# print((S1['p'].values).std())
# print(np.std(S1['p']))

# print(StanD(x))
# print(np.std(x))
# print(np.sqrt(np.mean(np.abs(x - np.mean(x))**2)))

In [36]:
print('average stable power is %f, average stable reactive is %f' % (avP,avR))

i=0
for S in [S1,S2,S3,S4]:
    i = i+1 
    print('window %i \n' % i)
    S['np'] = np.abs(S['p']-avP)
    S['nq'] = np.abs(S['q']-avR)
    
    
    print('mean active: %f \n' % S['np'].values.mean())
    print('min active: %f \n' % S['np'].values.min())
    print('max active: %f \n' % S['np'].values.max())
    print('std active: %f \n' % StanD(S['np'].values,S.shape[0]))
#     print('std active: %f \n'% S['np'].values.std())
    
    print('mean reactive: %f \n' % S['nq'].values.mean())
    print('min reactive: %f \n' % S['nq'].values.min())
    print('max reactive: %f \n' % S['nq'].values.max())
    print('std reactive: %f \n'% S['nq'].values.std())

average stable power is -0.037500, average stable reactive is 2.257500
window 1 

mean active: 335.092500 

min active: 252.037500 

max active: 373.637500 

std active: 34.134147 

mean reactive: 3.340750 

min reactive: 0.057500 

max reactive: 7.042500 

std reactive: 1.854537 

window 2 

mean active: 144.370000 

min active: 86.037500 

max active: 244.437500 

std active: 47.070077 

mean reactive: 4.220000 

min reactive: 0.242500 

max reactive: 6.842500 

std reactive: 1.863127 

window 3 

mean active: 71.107500 

min active: 64.037500 

max active: 84.737500 

std active: 5.935495 

mean reactive: 2.177250 

min reactive: 0.042500 

max reactive: 6.742500 

std reactive: 1.599297 

window 4 

mean active: 61.797500 

min active: 60.337500 

max active: 63.837500 

std active: 1.011385 

mean reactive: 1.557500 

min reactive: 0.142500 

max reactive: 3.442500 

std reactive: 0.939295 



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  after removing the cwd from sys.path.


In [8]:
df['diff'] = df['sN'] - df['sN'].shift()
# df = df.dropna()

In [59]:
df['p'].iloc[72:122].mean()

0.56

In [55]:
df2

,sN,ms,P,S,avgP,minP,maxP,stdP,avgR,minR,maxR,stdR
0,8169,458463.0,1.0,1.0,3.9,3.9,4.0,163.3,8.3,8.3,8.3,8.3
1,8175,458513.0,1.0,2.0,3.9,3.9,4.1,163.3,8.1,8.1,8.3,8.3
2,8181,458564.0,1.0,3.0,4.0,4.0,4.3,163.3,8.1,8.1,8.3,8.1
3,8187,458614.0,1.0,4.0,4.0,4.0,4.1,163.3,8.1,8.1,8.3,8.3
